In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import io
import pickle
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from itertools import combinations, chain
from torch.utils.data import TensorDataset, DataLoader
from collections import Counter

In [2]:
gauth = GoogleAuth()
gauth.DEFAULT_SETTINGS['client_config_file'] = 'client_secret_1057507276332-5mk9ac9q22rsmtm1idlqvpraq08ar8p5.apps.googleusercontent.com.json'
gauth.LoadCredentialsFile("mycreds.txt")
if gauth.credentials is None:
    gauth.LocalWebserverAuth()
elif gauth.access_token_expired:
    gauth.Refresh()
else:
    gauth.Authorize()

gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

In [3]:
max_games = 500000 
asset_dir = 'asset'
file_name = '2023_tc_50000_games.pgn'

cached_urls_file = file_name.split('.')[0] + '_urls_list.pkl'
cached_ratings_file = file_name.split('.')[0] + '_ratings_list.pkl'
cached_games_file = file_name.split('.')[0] + '_game_arrays.pkl'

In [4]:
def find_folder_id(folder_name):
    """Find and return the Google Drive folder ID for a given folder name."""
    file_list = drive.ListFile({'q': f"title='{folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()
    for file in file_list:
        if file['title'] == folder_name:
            return file['id']
    return None

def read_pkl_file_from_drive(file_title, parent_id):
    """Read a .pkl file directly from Google Drive into a Python object."""
    query = f"'{parent_id}' in parents and trashed=false and title='{file_title}'"
    file_list = drive.ListFile({'q': query}).GetList()
    if not file_list:
        print(f"No file found with title: {file_title}")
        return None
    file = file_list[0]
    file_content = file.GetContentString(encoding='cp437')
    buffer = io.BytesIO(file_content.encode('cp437'))
    return pickle.load(buffer)

asset_folder_id = find_folder_id(asset_dir)
if asset_folder_id is None:
    print("Asset folder not found.")
else:
    file_titles = {
        'urls_list': cached_urls_file,
        'ratings_list': cached_ratings_file,
        'game_arrays': cached_games_file,
    }

    urls_list = read_pkl_file_from_drive(file_titles['urls_list'], asset_folder_id)
    ratings_list = read_pkl_file_from_drive(file_titles['ratings_list'], asset_folder_id)
    game_arrays = read_pkl_file_from_drive(file_titles['game_arrays'], asset_folder_id)

    if urls_list is not None:
        print("URLs list loaded successfully.")
    if ratings_list is not None:
        print("Ratings list loaded successfully.")
    if game_arrays is not None:
        print("Game arrays loaded successfully.")

URLs list loaded successfully.
Ratings list loaded successfully.
Game arrays loaded successfully.


In [5]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout_rate=0):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, hidden_size)
        self.fc6 = nn.Linear(hidden_size, hidden_size)
        self.fc7 = nn.Linear(hidden_size, hidden_size)
        self.fc_classification = nn.Linear(hidden_size, num_classes)
        self.fc_regression = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 

        out, _ = self.lstm(x, (h0,c0))  
        out = out[:, -1, :]
        
        out = self.dropout(F.relu(self.fc1(out)))
        out = self.dropout(F.relu(self.fc2(out)))
        out = self.dropout(F.relu(self.fc3(out)))
        out = self.dropout(F.relu(self.fc4(out)))
        out = self.dropout(F.relu(self.fc5(out)))
        out = self.dropout(F.relu(self.fc6(out)))
        out = self.dropout(F.relu(self.fc7(out)))
        classification_output = self.fc_classification(out)
        regression_output = self.fc_regression(out)
        return classification_output, regression_output

In [6]:
def combined_loss(classification_output, regression_output, target, alpha=0.5):
    classification_loss = nn.CrossEntropyLoss()(classification_output, target)
    regression_target = target.float()
    regression_loss = nn.MSELoss()(regression_output.squeeze(), regression_target)
    return alpha * classification_loss + (1 - alpha) * regression_loss

def train_model(model, train_loader, test_loader, optimizer, num_epochs, device, alpha=0.5):
    torets = []
    for epoch in range(num_epochs):
        model.train()
        for i, (moves, labels) in enumerate(train_loader):  
            moves = moves.to(device)
            labels = labels.to(device)

            classification_output, regression_output = model(moves)
            loss = combined_loss(classification_output, regression_output, labels, alpha)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        #print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
        predicted_probs, predicted_labels, actual_labels = test_model(model, test_loader, device)
        pred_closeness = [sum(abs(p - a) <= k for p, a in zip(predicted_labels, actual_labels)) for k in range(10)]
        toret = [x/20000 for x in pred_closeness]
        torets.append(toret)
    return torets

def test_model(model, test_loader, device):
    model.eval()
    n_correct = 0
    n_samples = 0
    predicted_probs = []
    predicted_labels = []
    actual_labels = []
    with torch.no_grad():
        for moves, labels in test_loader:
            moves = moves.to(device)
            labels = labels.to(device)
            classification_output, _ = model(moves)
            probabilities = F.softmax(classification_output, dim=1)

            _, predicted = torch.max(classification_output.data, 1)
            predicted_probs.extend(probabilities.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())
            actual_labels.extend(labels.cpu().numpy())
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    #print(f'Accuracy of the network on the test moves: {acc} %')
    return predicted_probs, predicted_labels, actual_labels

In [7]:
def pad_game(game, max_length=256, vector_size=42):
    padding_length = max_length - len(game)
    if padding_length < 0:
        return game[:max_length]
    else:
        padding = np.full((padding_length, vector_size), -1)
        return np.vstack((game, padding))

In [8]:
def get_loaders(padded_games, ratings_list, urls_list, batch_size, fold_number=0):
    if fold_number < 0 or fold_number > 4:
        raise ValueError("fold_number must be between 0 and 4")
    test_list = padded_games[fold_number::5]
    #print(len(test_list))
    train_list = [df for i in range(5) if i != fold_number for df in padded_games[i::5]]
    test_ratings = ratings_list[fold_number::5]
    train_ratings = [ratings for i in range(5) if i != fold_number for ratings in ratings_list[i::5]]
    test_urls = urls_list[fold_number::5]
    train_urls = [url for i in range(5) if i != fold_number for url in urls_list[i::5]]

    train_data = [torch.FloatTensor(doc) for doc in train_list]
    test_data = [torch.FloatTensor(doc) for doc in test_list]
    train_labels = torch.LongTensor(train_ratings)
    test_labels = torch.LongTensor(test_ratings)

    train_dataset = TensorDataset(torch.stack(train_data), train_labels)
    test_dataset = TensorDataset(torch.stack(test_data), test_labels)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader, train_urls, test_urls

In [9]:
input_size = 42
hidden_size = 100
num_classes = 10
num_epochs = 18
num_layers = 2
learning_rate = 0.001
dropout_rate = 0.25
sequence_length = 80
batch_size = 80
alpha = 0.8
decay = 0.0000064

torch.manual_seed(64)

In [10]:
piece_indices = [0, 1, 2, 4, 5, 9, 10, 11, 12, 14, 15, 27, 28, 29, 30, 31, 32] #17
domain_indices = [13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26] #12
time_indices = [6, 7, 8, 33, 34, 35, 36] #7
engine_indices = [3, 37, 38, 39, 40, 41] #6
type_inds = [piece_indices, domain_indices, time_indices, engine_indices]
ind_names = ['piece_indices', 'domain_indices', 'time_indices', 'engine_indices']

for r in range(1, len(type_inds) + 1):
    for combo in combinations(enumerate(type_inds), r):
        keep_indices = list(chain(*[c[1] for c in combo]))
        names = [ind_names[c[0]] for c in combo]

        input_size = len(keep_indices)
        game_arrays_trunc = [arr[:, keep_indices] for arr in game_arrays]
        padded_games = [pad_game(g, sequence_length, input_size) for g in game_arrays_trunc]
        
        train_loader, test_loader, train_urls, test_urls = get_loaders(padded_games, ratings_list, urls_list, batch_size)
        model_path = file_name.split('.')[0] + '_pred.pth'
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=decay)
        num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f'Model: {names}, Parameters: {num_params:,}')
        #model.load_state_dict(torch.load(model_path, map_location=device))
        #model.to(device)

        lists = train_model(model, train_loader, test_loader, optimizer, num_epochs, device, alpha)
        predicted_probs, predicted_labels, actual_labels = test_model(model, test_loader, device)

        pred_closeness = [sum(abs(p - a) <= k for p, a in zip(predicted_labels, actual_labels)) for k in range(10)]
        print([x/20000 for x in pred_closeness])

        truths_counter = Counter(actual_labels)
        preds_counter = Counter(predicted_labels)
        actual_df = pd.DataFrame(list(truths_counter.items()), columns=['Label', 'Actual Count'])
        predicted_df = pd.DataFrame(list(preds_counter.items()), columns=['Label', 'Predicted Count'])
        merged_df = pd.merge(actual_df, predicted_df, on='Label', how='outer').fillna(0)
        sorted_df = merged_df.set_index('Label').sort_index()
        corrects_list = []
        recall_list = []
        for i in range(10):
            indices = [index for index, value in enumerate(actual_labels) if value == i]
            totals = [predicted_labels[ind] for ind in indices]
            corrects = [value for value in totals if value == i]
            rate = len(corrects)/len(totals)
            corrects_list.append(len(corrects))
            recall_list.append(rate)

        sorted_df['Correct Predictions'] = corrects_list
        sorted_df['Identify Rate'] = pd.Series(recall_list).round(3)
        display(sorted_df.T)
        print()

Model: ['piece_indices'], Parameters: 200,211
[0.2435, 0.61335, 0.81845, 0.9253, 0.97405, 0.99115, 0.99715, 0.99935, 0.99995, 1.0]


Label,0,1,2,3,4,5,6,7,8,9
Actual Count,1625.000,2149.000,2091.00,2043.000,2026.0,2116.000,2157.000,2114.000,2059.000,1620.000
Predicted Count,1580.000,2421.000,2661.00,2506.000,0.0,4295.000,2202.000,1365.000,2318.000,652.000
Correct Predictions,672.000,704.000,564.00,484.000,0.0,781.000,402.000,288.000,681.000,294.000
Identify Rate,0.414,0.328,0.27,0.237,0.0,0.369,0.186,0.136,0.331,0.181



Model: ['domain_indices'], Parameters: 198,211
[0.2278, 0.5759, 0.784, 0.89575, 0.96075, 0.9853, 0.99445, 0.99915, 0.99985, 1.0]


Label,0,1,2,3,4,5,6,7,8,9
Actual Count,1625.000,2149.000,2091.000,2043.0,2026.000,2116.000,2157.0,2114.000,2059.000,1620.00
Predicted Count,777.000,3121.000,3082.000,0.0,1662.000,5144.000,0.0,3661.000,1272.000,1281.00
Correct Predictions,414.000,919.000,624.000,0.0,271.000,842.000,0.0,716.000,349.000,421.00
Identify Rate,0.255,0.428,0.298,0.0,0.134,0.398,0.0,0.339,0.169,0.26



Model: ['time_indices'], Parameters: 196,211
[0.19535, 0.49265, 0.68185, 0.82715, 0.9053, 0.95475, 0.97985, 0.993, 0.99965, 1.0]


...

Label,0,1,2,3,4,5,6,7,8,9
Actual Count,1625.000,2149.000,2091.000,2043.000,2026.000,2116.000,2157.000,2114.000,2059.000,1620.000
Predicted Count,1291.000,2934.000,1912.000,2079.000,2184.000,1210.000,2802.000,1205.000,2015.000,2368.000
Correct Predictions,687.000,990.000,496.000,484.000,514.000,280.000,678.000,308.000,581.000,999.000
Identify Rate,0.423,0.461,0.237,0.237,0.254,0.132,0.314,0.146,0.282,0.617



Model: ['domain_indices', 'time_indices', 'engine_indices'], Parameters: 203,411
[0.29195, 0.69045, 0.88405, 0.9662, 0.9914, 0.9977, 0.99995, 1.0, 1.0, 1.0]


Label,0,1,2,3,4,5,6,7,8,9
Actual Count,1625.000,2149.000,2091.000,2043.000,2026.000,2116.000,2157.000,2114.0,2059.000,1620.00
Predicted Count,1173.000,2374.000,2349.000,1557.000,1998.000,450.000,5714.000,0.0,2474.000,1911.00
Correct Predictions,677.000,862.000,623.000,353.000,429.000,83.000,1243.000,0.0,711.000,858.00
Identify Rate,0.417,0.401,0.298,0.173,0.212,0.039,0.576,0.0,0.345,0.53



Model: ['piece_indices', 'domain_indices', 'time_indices', 'engine_indices'], Parameters: 210,211
[0.31845, 0.7371, 0.9139, 0.97605, 0.9938, 0.99845, 0.99965, 1.0, 1.0, 1.0]


Label,0,1,2,3,4,5,6,7,8,9
Actual Count,1625.000,2149.000,2091.000,2043.000,2026.00,2116.00,2157.000,2114.000,2059.000,1620.000
Predicted Count,1187.000,2040.000,2161.000,2413.000,956.00,2731.00,1882.000,2856.000,1862.000,1912.000
Correct Predictions,705.000,776.000,621.000,624.000,244.00,657.00,468.000,754.000,620.000,900.000
Identify Rate,0.434,0.361,0.297,0.305,0.12,0.31,0.217,0.357,0.301,0.556
